In [2]:
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifier
from textgrad.loss import TextLoss

### Setup LLM Engine

In [3]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro")

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup Solution to Optimize

In [4]:
initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

solution = Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")


### Set Loss Variable

In [5]:
loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
Do not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise.""",
                              requires_grad=False,
                              role_description="system prompt")


### Inisiate Optimizer

In [6]:
optimizer = TextualGradientDescent([solution])

### Find Loss

In [7]:
loss = TextLoss(loss_system_prompt, engine=engine)
result = loss(solution) # Forward method in Loss Function

print("INITIAL LOSS:", result)

INITIAL LOSS: The calculation of  b² - 4ac is incorrect: it should be (-7)² - 4 * 3 * 2 = 49 - 24 = 25, not 49 + 24 = 73.  Also, the denominator is missing in the final answers for x1 and x2; it should be 6 in both cases.



### Using TextualVerifier to Verify Loss

In [8]:
# Verify Loss
verifier = TextualVerifier(verifier_engine=engine, step_eval_iterations=3, logger=False)
verified_result = verifier.verify(instance=solution, 
                                    prompt=loss_system_prompt,
                                    calculation=result)

print("VERIFIED RESULT:", verified_result.value)

VERIFIED RESULT: The core issue across these evaluations is a lack of clear, systematic process for calculating the discriminant and applying the quadratic formula, leading to sign errors and misinterpretation of the formula itself.  A robust process should emphasize the following:

1. **Explicit Formula Statement:** Always begin by explicitly stating the quadratic formula *and* the discriminant formula separately. This reinforces the correct structure and prevents misremembered or combined formulas.

2. **Clear Coefficient Identification:** Explicitly identify the values of a, b, and c, *including their signs*, before any calculations. This prevents errors stemming from incorrect substitutions, especially with negative coefficients.

3. **Parentheses for Substitution:**  Substitute the values of a, b, and c into the discriminant formula *using parentheses*. This is crucial for maintaining the correct order of operations and preventing sign errors, particularly when squaring negative n

### Optimize Solution with Optimizer (TGD)

In [9]:
# TRANSFER: Replace the loss value with verified result
result.set_value(verified_result.value) 

print("FINAL LOSS (after transfer):", result.value)

FINAL LOSS (after transfer): The core issue across these evaluations is a lack of clear, systematic process for calculating the discriminant and applying the quadratic formula, leading to sign errors and misinterpretation of the formula itself.  A robust process should emphasize the following:

1. **Explicit Formula Statement:** Always begin by explicitly stating the quadratic formula *and* the discriminant formula separately. This reinforces the correct structure and prevents misremembered or combined formulas.

2. **Clear Coefficient Identification:** Explicitly identify the values of a, b, and c, *including their signs*, before any calculations. This prevents errors stemming from incorrect substitutions, especially with negative coefficients.

3. **Parentheses for Substitution:**  Substitute the values of a, b, and c into the discriminant formula *using parentheses*. This is crucial for maintaining the correct order of operations and preventing sign errors, particularly when squarin

In [10]:
# Optimize
result.backward()

optimizer.step()
print("SOLUTION:", solution.value)

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \nDo not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:\nx = (-b ± √(b^2 - 4ac)) / 2a\na = 3, b = -7, c = 2\nx = (7 ± √((-7)^2 + 4(3)(2))) / 6\nx = (7 ± √73) / 6\nThe solutions are:\nx1 = (7 + √73)\nx2 = (7 - √73) </LM_INPUT>\n\n<LM_OUTPUT> The core issue across these evaluations is a lack of clear, systematic process for calculating the discriminant and applying the quadratic formula, leading to sign errors and misinterpretation of the formula itself.  A robust process should emphasize the following:\n\n1. **Explicit Formula Statement:** Always begin by explicitly stating the quadratic formula *and* the discriminant formula separately. This reinforces the correct structure and prevents misremembered or combined formulas.\n\n2.